In [1]:
import time
import threading
import warnings
from datetime import datetime
from typing import Dict , Optional
import pandas as pd # type: ignore
warnings.filterwarnings('ignore')

# interactive broker API live

In [2]:
!pip install ibapi
!pip install ib_insync

DEPRECATION: Loading egg at /opt/anaconda3/envs/stock/lib/python3.11/site-packages/ibapi-10.30.1-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/envs/stock/lib/python3.11/site-packages/ibapi-10.30.1-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
from ibapi.client import EClient # type: ignore
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.common import BarData

In [4]:
class TradingApp(EClient,EWrapper):
    def __init__(self)->None:
        EClient.__init__(self,self)
        self.data: Dict[int,pd.DataFrame]={}
    def error(self,reqId:int, erroCode:int,errorString:str)->None:
        print(f'Error: ',{reqId},{erroCode},{errorString})
    def get_historical_data(self,reqId:int,contract:Contract)->pd.DataFrame:
        self.data[reqId]=pd.DataFrame(columns=['time','high','low','close'])
        self.data[reqId].set_index('time',inplace=True)
        self.reqHistoricalData(
            reqId=reqId,
            contract=contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting='1 min',
            whatToShow='MIDPOINT',
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[],
        )
        time.sleep(4)
        return self.data[reqId]
    def historicalData(self, reqId:int, bar:BarData)->None:
        df=self.data[reqId]
        df.loc[pd.to_datetime(bar.date,unit='s'),['high','low','close']]=[bar.high,bar.low,bar.close]
        df=df.astype(float)
        self.data[reqId]=df
    @staticmethod
    def get_stock_contract(symbol:str)->Contract:
        contract=Contract()
        contract.symbol=symbol
        contract.secType='STK'
        contract.exchange='SMART'
        contract.currency='USD'
        return contract
        
    
        


# you need to have TWS API and Trader WorkStation

In [5]:
app=TradingApp()


In [6]:
app.connect('127.0.0.1',7497,clientId=6)

In [7]:
threading.Thread(target=app.run,daemon=True).start()

Error:  {-1} {2104} {'Market data farm connection is OK:usfarm.nj'}
Error:  {-1} {2104} {'Market data farm connection is OK:usfuture'}
Error:  {-1} {2104} {'Market data farm connection is OK:eufarm'}
Error:  {-1} {2104} {'Market data farm connection is OK:cafarm'}
Error:  {-1} {2104} {'Market data farm connection is OK:usfarm'}
Error:  {-1} {2106} {'HMDS data farm connection is OK:ushmds'}
Error:  {-1} {2158} {'Sec-def data farm connection is OK:secdefnj'}


In [8]:
nvda=TradingApp.get_stock_contract('NVDA')

In [9]:
nvda

4593249168: 0,NVDA,STK,,0.0,,,SMART,,USD,,,False,,combo:

In [10]:
data=app.get_historical_data(99,nvda)
data

,high,low,close
time,,,
2025-03-28 08:00:00,118.07,110.98,111.14
2025-03-28 08:01:00,111.16,111.02,111.03
2025-03-28 08:02:00,111.17,111.00,111.13
2025-03-28 08:03:00,111.24,111.13,111.23
2025-03-28 08:04:00,111.33,111.22,111.29
...,...,...,...
2025-03-28 23:55:00,109.02,109.00,109.02
2025-03-28 23:56:00,109.06,109.01,109.06
2025-03-28 23:57:00,109.08,109.03,109.03


## These are most useful libraries in python for indicators and chart for data . 
### pip install mplfinance for renko and kagi
## pip install pandas_ta for (indicator like MACD,RSI ATR,..) https://github.com/twopirllc/pandas-ta.git
# pip install plotly  for better visualization, like Kagi and Renko